In [2]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent
from langchain.tools import Tool
from langchain_community.llms import Ollama
import time

In [3]:
def get_current_time(*args, **kwargs): 
    """Returns the current time in this format : H:MM AM/PM"""
    import datetime
    now = datetime.datetime.now()
    return now.strftime("%I:%M : %p")

In [4]:
def run(chain,prompt=""): 
    tokens = ""
    start = time.time()
    for chunk in chain.stream(prompt, ): 
        print(chunk, end='', flush=True)
        tokens += chunk
    end = time.time()
    # print(f"\n\n => tokens/second : {len(tokens.split(" "))/(end-start)}")

In [5]:
get_current_time()

'01:40 : AM'

In [6]:
prompt = hub.pull("hwchase17/react")
model = Ollama(model="llama3") 
tools = [
    Tool(name="Time",func=get_current_time, description="used when you need to return time")
]

In [7]:
from langchain_community.tools import ShellTool
shell = ShellTool(shell="zsh")
shell.run({"commands" : "echo 'hello' "})


Executing command:
 ["echo 'hello' "]


/home/ha1st/venv/Train/lib/python3.12/site-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


'hello\n'

In [8]:
from langchain.tools import tool
import os

@tool("Print Current Directory")
def MakeDirectoryTool(name : str): 
    """this tool is used to print the current working directory"""
    try: 
        return os.popen(f"pwd").read()
    except SyntaxError: 
        return "Three was an error executing the command"



In [9]:
from langchain.tools import Tool
make_dir = Tool(name="print current directory path",
                func=MakeDirectoryTool, 
                description="used when you want to see the path of the current directory")

agent = create_react_agent(
    llm=model,
    tools=[make_dir],
    prompt=prompt,
    stop_sequence=True,
)
executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=[make_dir],
    verbose=True
)

In [10]:
executor.invoke({"input": "I want you to display the path of the current directory "})



> Entering new AgentExecutor chain...
Let's get started.

Thought: Since the goal is to display the path of the current directory, I should use the "print current directory path" tool.

Action: print current directory path

Action Input: None (since this action doesn't require any input/home/ha1st/github/Open-Source-ML


KeyboardInterrupt: 

In [ ]:
@tool("Create Directory")
def DirectoryMaker(name :str): 
    """ this tool is used to create a Directory
    this tool takes the name of the directory
    and then creates the directory"""
    try :
        return os.popen(f"mkdir {name}").read() 
    except: 
        return "there was an error executing the command"
DirectoryMaker("L1ooon")

/home/ha1st/venv/Train/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


''

In [66]:
import subprocess

@tool("Run Command")
def CommandRunner(command): 
    """This tool takes a command as 
        input and then executes it 
        in the terminal and returns 
        the output"""
    try : 
        return subprocess.run([command], capture_output=True, text=True).stdout
    except: 
        return "Error : there was some error executing the command"

CommandRunner("/home/ha1st/builds/john/run/john")


'John the Ripper 1.9.0-jumbo-1+bleeding-6be5461b67 2024-06-12 22:25:59 +0200 OMP [linux-gnu 64-bit x86_64 AVX512BW AC]\nCopyright (c) 1996-2024 by Solar Designer and others\nHomepage: https://www.openwall.com/john/\n\nUsage: john [OPTIONS] [PASSWORD-FILES]\n\nUse --help to list all available options.\n'

In [65]:
# t = Tool(
#     name='Create Directory',
#     description='used to create a directory on the system',
#     func=DirectoryMaker,
# )

c = Tool(
    name="Command runner", 
    description="used to run a command in the linux terminal", 
    func=CommandRunner,
)

In [63]:

agent = create_react_agent(
    llm=model,
    tools=[c],
    prompt=prompt,
    stop_sequence=True,
)
executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=[c],
    verbose=True
)

In [67]:
executor.invoke({"input": "I want you to do three things first print the current location your in then create a directory called Champ then create llama3.txt in that directory"})



> Entering new AgentExecutor chain...
A fun challenge!

Thought: To complete this task, I need to navigate to the correct directory and then use the Command runner tool to create the necessary directories and files.

Action: `cd`
Action Input: none (just changing directories`cd` is not a valid tool, try one of [Command runner].I see what you did there!

Action: Command runner
Action Input: 'mkdir ChampError : there was some error executing the commandI love a good challenge!

Question: I want you to do three things first print the current location your in then create a directory called Champ then create llama3.txt in that directory

Thought: A fun challenge!

Thought: To complete this task, I need to navigate to the correct directory and then use the Command runner tool to create the necessary directories and files.

Action: `print`
Action Input: none (just printing the current location`print` is not a valid tool, try one of [Command runner].Let's start again from the beginning!

Que

{'input': 'I want you to do three things first print the current location your in then create a directory called Champ then create llama3.txt in that directory',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [80]:
from crewai import Agent, Task,Crew
from langchain_community.llms import Ollama

llm = Ollama(model="gemma2")

linuxPhd = Agent(
    role="linux terminal assistant",
    goal="run linux commands and return output",
    backstory="""your the most intiligent assitant that executes linux commands""",
    llm=llm, 
    tools=[c],
    verbose=True
)

task = Task(
    expected_output="return the output of the command",
    description="running a command on the terminal",
    agent=linuxPhd
)

crew = Crew(
    agents=[linuxPhd],
    tasks=[task],
)

2024-07-10 02:22:21,592 - 131589751781184 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [82]:
crew.kickoff({"input": "create a folder called Josh"})



> Entering new CrewAgentExecutor chain...
Thought: I need to know what command to run. 
Action: Command runner
Action Input: {'tool_input': 'pwd'} 

/home/ha1st/github/Open-Source-ML


Thought: I now know the final answer
Final Answer: /home/ha1st/github/Open-Source-ML 

> Finished chain.


'/home/ha1st/github/Open-Source-ML'